In [13]:
import time
from datetime import datetime
import requests
from ugot import ugot

# Initialize Ugot bot
bot = ugot.UGOT()
bot.initialize('192.168.135.49')  # Replace with your robot's IP address

# Configuration
ACTIVATION_PHRASE = "hi robot"
ROBOT_GREETING = "Hi! What's up?"
VOICE_TYPE = 0  # 0: female, 1: male
WEATHER_API_KEY = "ce89e1cabdee39be7f06489a3fe7d232"  # OpenWeatherMap API key
NEWS_API_KEY = "9769065b683b483c97a987fb6c10f331"  # Replace with your NewsAPI key
WEATHER_LOCATION = "India"  # Change as needed
NEWS_COUNTRY = "in"  # 'in' for India; use 'us' for the USA, etc.

# Utility Functions
def get_current_time():
    """Returns the current time in a human-readable format."""
    return datetime.now().strftime("%I:%M %p")

def get_current_date():
    """Returns the current date in a human-readable format."""
    return datetime.now().strftime("%A, %d %B %Y")

def get_weather():
    """Fetches weather information using OpenWeatherMap API."""
    url = f"http://api.openweathermap.org/data/2.5/weather?q={WEATHER_LOCATION}&appid={WEATHER_API_KEY}&units=metric"
    try:
        response = requests.get(url)
        data = response.json()
        if data.get("cod") == 200:
            temp = data["main"]["temp"]
            weather_desc = data["weather"][0]["description"]
            return f"The current temperature in {WEATHER_LOCATION} is {temp}°C with {weather_desc}."
        else:
            return "Sorry, I couldn't fetch the weather information."
    except Exception as e:
        return "There was an error retrieving the weather data."

def get_news():
    """Fetches the latest news headlines using NewsAPI."""
    url = f"https://newsapi.org/v2/top-headlines?country={NEWS_COUNTRY}&apiKey={NEWS_API_KEY}"
    try:
        response = requests.get(url)
        data = response.json()
        if data.get("status") == "ok":
            articles = data.get("articles", [])
            if articles:
                headline = articles[0]["title"]
                return f"Here is the latest news: {headline}."
            else:
                return "I couldn't find any news articles at the moment."
        else:
            return "Sorry, I couldn't fetch the news."
    except Exception as e:
        return "There was an error retrieving the news."

# Infinite Loop to Process User Inputs
while True:
    try:
        audio = bot.start_audio_asr()  # Start mic and get user input
        if audio and ACTIVATION_PHRASE.lower() in audio.lower():  # Check activation
            bot.play_audio_tts(str(ROBOT_GREETING), wait=True, voice_type=VOICE_TYPE)
            time.sleep(0.5)  # Allow greeting to complete

            while True:
                query = bot.start_audio_asr().strip()  # Get user query

                if not query:
                    continue  # If query is empty, skip iteration
                
                # Handle queries
                if "time" in query.lower():
                    response = f"The current time is {get_current_time()}."
                elif "date" in query.lower():
                    response = f"Today's date is {get_current_date()}."
                elif "weather" in query.lower():
                    response = get_weather()
                elif "news" in query.lower():
                    response = get_news()
                elif query.lower() in ["exit", "stop", "shutdown"]:
                    response = "Goodbye! Shutting down."
                    bot.play_audio_tts(str(response), wait=True, voice_type=VOICE_TYPE)
                    print("Exiting program...")
                    exit(0)
                else:
                    # Fallback to AI NLP for other queries
                    response = bot.start_audio_nlp(query, wait=True)

                # Ensure response is always a string before passing to TTS
                bot.play_audio_tts(str(response), wait=True, voice_type=VOICE_TYPE)
                time.sleep(0.5)  # Allow processing time

    except KeyboardInterrupt:
        print("Program interrupted. Exiting...")
        bot.play_audio_tts("Goodbye! Shutting down.", wait=True, voice_type=VOICE_TYPE)
        break
    except Exception as e:
        print(f"Error: {e}")
        with open("error_log.txt", "a") as log_file:
            log_file.write(f"{datetime.now()} - Error: {e}\n")
        bot.play_audio_tts("An error occurred. Check logs for details.", wait=True, voice_type=VOICE_TYPE)


192.168.135.49:50051
